In [1]:
import pandas as pd
class2 = pd.read_csv('../080289-main/chap10/data/class2.csv')

from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
onehot_encoder = preprocessing.OneHotEncoder()

train_x = label_encoder.fit_transform(class2['class2'])
train_x

array([2, 2, 1, 0, 1, 0])

## 카운터 벡터(counter Vector)

In [2]:
from sklearn.feature_extraction.text import CountVectorizer

corpus = ["When I'm away from you, I miss your touch (ooh)",
        "You're the reason I believe in love",
        "It's been difficult for me to trust (ooh)",
        "And I'm afraid that I'ma fuck it up",
        "Ain't no way that I can leave you stranded",
        "'Cause you ain't never left me empty-handed",
        "And you know that I know that I can't live without you"]
vectorizer = CountVectorizer()
vectorizer.fit(corpus)
vectorizer.vocabulary_

{'when': 37,
 'away': 3,
 'from': 11,
 'you': 39,
 'miss': 23,
 'your': 40,
 'touch': 33,
 'ooh': 26,
 're': 27,
 'the': 31,
 'reason': 28,
 'believe': 5,
 'in': 14,
 'love': 20,
 'it': 15,
 'been': 4,
 'difficult': 8,
 'for': 10,
 'me': 22,
 'to': 32,
 'trust': 34,
 'and': 2,
 'afraid': 0,
 'that': 30,
 'ma': 21,
 'fuck': 12,
 'up': 35,
 'ain': 1,
 'no': 25,
 'way': 36,
 'can': 6,
 'leave': 17,
 'stranded': 29,
 'cause': 7,
 'never': 24,
 'left': 18,
 'empty': 9,
 'handed': 13,
 'know': 16,
 'live': 19,
 'without': 38}

In [3]:
vectorizer.transform(["Ain't no way that I can leave you stranded"]).toarray()

array([[0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0]],
      dtype=int64)

In [6]:
# 불용어 제거 countVector
vectorizer = CountVectorizer(stop_words=['the', 'that', 'ooh']).fit(corpus)
vectorizer.vocabulary_

{'when': 34,
 'away': 3,
 'from': 11,
 'you': 36,
 'miss': 23,
 'your': 37,
 'touch': 30,
 're': 26,
 'reason': 27,
 'believe': 5,
 'in': 14,
 'love': 20,
 'it': 15,
 'been': 4,
 'difficult': 8,
 'for': 10,
 'me': 22,
 'to': 29,
 'trust': 31,
 'and': 2,
 'afraid': 0,
 'ma': 21,
 'fuck': 12,
 'up': 32,
 'ain': 1,
 'no': 25,
 'way': 33,
 'can': 6,
 'leave': 17,
 'stranded': 28,
 'cause': 7,
 'never': 24,
 'left': 18,
 'empty': 9,
 'handed': 13,
 'know': 16,
 'live': 19,
 'without': 35}

## TF-IDF(Term Frequency-Inverse Document Frequency)

- TF(Term Frequency) 

문서 내에서 특정 단어가 출연한 빈도

- IDF(Inverse Document Frequency)

`DF(Document Frequency)`는 한 단어가 전체 문서에서 얼마나 공통적으로 많이 등장하는지 나타내는 값. 즉, 특정 단어가 나타난 문서 개수
특정 단어 t가 모든 문서에 등장하는 일반적인 단어(a, the, that 등)라면 TF-IDF 가중치를 낮추어 주어야 함. 
따라서 DF 값이 클수록 TF-IDF 값을 낮추기 위해 DF 값에 역수를 취하고, 이것이 IDF 값이 됨.

역수를 취하면 전체 문서 개수가 많아질수록 IDF 값도 커지므로, IDF 값은 로그(log)를 취해야 함.

이 때 전체 문서에 특정 단어가 발생하는 빈도가 0이라면, 분모가 0이 되는 상황이 발생함. 이를 방지하기 위해 
분모에 1을 더해 주는 것을 `스무딩(smoothing)`이라고 함.

In [10]:
# TF-IDF 적용하기
from sklearn.feature_extraction.text import TfidfVectorizer
doc = ['I love Choonsik', 'who love cats?', 'Choonsik is a cat', 'Cats are so lovely']
tfidf_vectorizer = TfidfVectorizer(min_df=1)
tfidf_matrix = tfidf_vectorizer.fit_transform(doc)
doc_dist = (tfidf_matrix * tfidf_matrix.T)
print(f'유사도 행렬 {doc_dist.get_shape()[0]} x {doc_dist.get_shape()[1]}:')
print(doc_dist.toarray())

유사도 행렬 4 x 4:
[[1.         0.37222485 0.34431452 0.        ]
 [0.37222485 1.         0.         0.21808385]
 [0.34431452 0.         1.         0.        ]
 [0.         0.21808385 0.         1.        ]]


## Word2Vec

Word2Vec은 신경망 알고리즘으로, 주어진 텍스트에서 텍스트 각 단어마다 하나씩 일련 벡터를 출력함
- 일정한 크기 윈도우(window)로 분할된 텍스트를 신경망 입력으로 사용함. 
- 이 때 모든 분할된 텍스트는 한 쌍의 대상 단어와 컨텍스트로 네트워크에 공급됨.
- 네트워크 은닉층에는 각 단어에 대한 가중치가 포함되어 있음

In [11]:
from nltk.tokenize import sent_tokenize, word_tokenize
import warnings
warnings.filterwarnings('ignore')
import gensim
from gensim.models import Word2Vec

sample = open('../080289-main/chap10/data/peter.txt', 'r', encoding='utf-8')
s = sample.read()

f = s.replace('\n', ' ')
data = []

for i in sent_tokenize(f) :
    temp = []
    for j in word_tokenize(i) :
        temp.append(j.lower())
    data.append(temp)
    
data

[['once',
  'upon',
  'a',
  'time',
  'in',
  'london',
  ',',
  'the',
  'darlings',
  'went',
  'out',
  'to',
  'a',
  'dinner',
  'party',
  'leaving',
  'their',
  'three',
  'children',
  'wendy',
  ',',
  'jhon',
  ',',
  'and',
  'michael',
  'at',
  'home',
  '.'],
 ['after',
  'wendy',
  'had',
  'tucked',
  'her',
  'younger',
  'brothers',
  'jhon',
  'and',
  'michael',
  'to',
  'bed',
  ',',
  'she',
  'went',
  'to',
  'read',
  'a',
  'book',
  '.'],
 ['she', 'heard', 'a', 'boy', 'sobbing', 'outside', 'her', 'window', '.'],
 ['he', 'was', 'flying', '.'],
 ['there', 'was', 'little', 'fairy', 'fluttering', 'around', 'him', '.'],
 ['wendy', 'opened', 'the', 'window', 'to', 'talk', 'to', 'him', '.'],
 ['“', 'hello', '!'],
 ['who', 'are', 'you', '?'],
 ['why', 'are', 'you', 'crying', '”', ',', 'wendy', 'asked', 'him', '.'],
 ['“', 'my', 'name', 'is', 'peter', 'pan', '.'],
 ['my',
  'shadow',
  'wouldn',
  '’',
  't',
  'stock',
  'to',
  'me.',
  '”',
  ',',
  'he',
  'rep

## CBOW(continuous bag of words)

CBOW는 단어를 여러 개 나열한 후 이와 관련된 단어를 추정하는 방식. 즉, 문장에서 등장하는 n개의 단어 열에서 
다음에 등장할 단어를 예측함.  

CBOW 벡터 크기는 은닉층 크기 N과 같음. 입력층과 은닉층 사이 가중치 W는 VXN 행렬이며,
은닉층에서 출력층 사이의 가중치 W'는 NXV 행렬임. 여기서 V는 단어 집합의 크기를 의미함.

In [21]:
# 데이터셋에 CBOW 적용

model = gensim.models.Word2Vec(data, min_count=1,
                              size=100, window=5, sg=0)
# min_count : 단어 최소 빈도수 제한, size : 임베딩 벡터 차원, window : 컨텍스트 윈도우 크기, sg : 0(CBOW,default), 1(skip-gram)

print('[CBOW]Cosine similarity :', model.wv.similarity('leaving', 'went'))
print('[CBOW]Cosine similarity :', model.wv.similarity('dinner', 'party'))

[CBOW]Cosine similarity : 0.09190975
[CBOW]Cosine similarity : 0.060172144


In [22]:
# 데이터셋에 skip-gram 적용

model = gensim.models.Word2Vec(data, min_count=1,
                              size=100,window=5,sg=1)
print('[skip-gram]Cosine similarity :', model.wv.similarity('leaving', 'went'))
print('[skip-gram]Cosine similarity :', model.wv.similarity('dinner', 'party'))

[skip-gram]Cosine similarity : 0.17475246
[skip-gram]Cosine similarity : 0.0945473


## FastText

워드투벡터 단점을 보완하고자 페이스북에서 개발한 임베딩 알고리즘.
기존 워드투벡터의 워드 임베딩 방식은 분산 표현(distributed representation)을 이용하여 단어 분산 분포가
유사한 단어들에 비슷한 벡터값을 할당하여 표현함. 따라서 워드투벡터는 사전에 없는 단어는 벡터 값을 얻을 수 없음.
또한, 자주 사용되지 않는 단어에 대해서 학습이 불안정함.

--> 패스트텍스트는 이러한 단점을 보환하고, word representation 방법을 사용함.
패스트텍스트는 노이즈에 강하며, 새로운 단어에 대해서 형태적 유사성을 고려한 벡터값을 얻기 때문에
자연어 처리 분야에서 많이 사용되는 알고리즘임

- **사전에 없는 단어에 벡터 값을 부여하는 방법**

주어진 문서의 각 단어를 n-gram으로 표현함. 이 때 n의 설정에 따라 단어의 분리 수준이 결정됨.

> n=3일때
> "This is Deep Learning Book" ==> [This is Deep, is Deep Learning, Deep Learning Book]으로 분리한 후 임베딩됨

패스트텍스트는 인공 신경망을 이용하여 학습이 완료된 후 데이터셋 모든 단어를 N-gram에 대해 임베딩함.
따라서 사전에 없는 단어가 등장하면 N-gram으로 분리된 부분 단어와 유사도를 계산하여 의미를 유추할 수 있음.

- **자주 사용되지 않는 단어에 학습 안정성을 확보하는 방법**

패스트텍스트는 등장 빈도수가 작더라도, n-gram으로 임베딩하기 때문에 참고할 수 있는 경우의 수가 많음.
따라서 상대적으로 자주 사용되지 않는 단어에서도 정확도가 높음

In [23]:
from gensim.test.utils import common_texts
from gensim.models import FastText

model = FastText('../080289-main/chap10/data/peter.txt', 
                size=4, window=3, min_count=1, iter=10)

In [25]:
sim_score = model.wv.similarity('peter', 'wendy')
print(sim_score)

0.26253965


In [27]:
sim_score = model.wv.similarity('peter','hook')
print(sim_score)

-0.15309206


In [31]:
# pre-trained fastText

from __future__ import print_function
from gensim.models import KeyedVectors

model_kr = KeyedVectors.load_word2vec_format('./wiki.ko.vec')

In [33]:
find_similar_to = '노력'

for similar_word in model_kr.similar_by_word(find_similar_to) :
    print(f'Word : {similar_word[0]}, Similarity : {similar_word[1]:.2f}')

Word : 노력함, Similarity : 0.80
Word : 노력중, Similarity : 0.75
Word : 노력만, Similarity : 0.72
Word : 노력과, Similarity : 0.71
Word : 노력의, Similarity : 0.69
Word : 노력가, Similarity : 0.69
Word : 노력이나, Similarity : 0.69
Word : 노력없이, Similarity : 0.68
Word : 노력맨, Similarity : 0.68
Word : 노력보다는, Similarity : 0.68


In [35]:
# positive / negative
similarities = model_kr.most_similar(positive=['동물', '육식'], negative=['사람'])
print(similarities)

[('육식동물', 0.7407333850860596), ('초식동물', 0.7166223526000977), ('거대동물', 0.6989486217498779), ('육식동물의', 0.6935194730758667), ('육상동물', 0.6817888021469116), ('동물기', 0.6815097332000732), ('반추동물', 0.67894446849823), ('육식성', 0.6774147748947144), ('육식동물로', 0.6773731708526611), ('유두동물', 0.6770071983337402)]


## GloVe(Gloval Vectors for Word Representation)

글로브는 횟수 기반의 LSA(Latent Semantic Analysis, 잠재의미 분석)와 예측 기반의 워드투벡터 단점을 보완하기 위한 모델임.

글로브는 단어에 대한 `동시 발생 확률(global co-occurence statistics)` 정보를 포함하는 단어 임베딩 방법.
즉, 단어에 대한 통계정보(통계기법)와 skip-gram을 합친 방식임. 글로브를 사용하면 단어 간 관련성을 통계적 방법으로 표현함.

In [37]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from sklearn.decomposition import PCA
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

In [45]:
glove_file = datapath('D:\\python\\self_study\\pyTorch\\NLP\\glove.6B\\glove.6B.100d.txt')
word2vec_glove_file = get_tmpfile('glove.6B.100d.txt')
glove2word2vec(glove_file, word2vec_glove_file)

(400000, 100)

In [46]:
# 유사한 단어 리스트 반환
model = KeyedVectors.load_word2vec_format(word2vec_glove_file)
model.most_similar('bill')

[('legislation', 0.8072140216827393),
 ('proposal', 0.7306863069534302),
 ('senate', 0.7142540812492371),
 ('bills', 0.7044401168823242),
 ('measure', 0.6958035230636597),
 ('passed', 0.6906244158744812),
 ('amendment', 0.6846879720687866),
 ('provision', 0.6845567226409912),
 ('plan', 0.6816462874412537),
 ('clinton', 0.6663139462471008)]

In [47]:
model.most_similar('simple')

[('easy', 0.7649385929107666),
 ('example', 0.751956582069397),
 ('rather', 0.7512185573577881),
 ('straightforward', 0.7487727403640747),
 ('sort', 0.7337545156478882),
 ('perfect', 0.7206918597221375),
 ('kind', 0.7189165353775024),
 ('simply', 0.710086464881897),
 ('sometimes', 0.7098439931869507),
 ('typical', 0.7076402902603149)]

In [48]:
model.most_similar(negative=['simple']) # 관련 없는 단어

[('numurkah', 0.5687417387962341),
 ('levi-montalcini', 0.5484330654144287),
 ('pansa', 0.5436047315597534),
 ('wutr', 0.5427923202514648),
 ('scodelario', 0.5364961624145508),
 ('3.7996', 0.5360996723175049),
 ('jalayirids', 0.5360099077224731),
 ('skradin', 0.5345335006713867),
 ('edirisinghe', 0.5324875116348267),
 ('methoni', 0.5324363708496094)]

In [49]:
model.most_similar(positive=['forest', 'tree'], negative=['city'])

[('trees', 0.7250508069992065),
 ('forests', 0.684791088104248),
 ('pine', 0.6578460335731506),
 ('deciduous', 0.6177636384963989),
 ('rainforest', 0.6123120784759521),
 ('vegetation', 0.6096251010894775),
 ('coniferous', 0.5996668934822083),
 ('canopy', 0.5959811806678772),
 ('habitat', 0.5957985520362854),
 ('mangrove', 0.5954805016517639)]

In [50]:
model.doesnt_match(['true', 'sincere', 'honest', 'conscience', 'lie']) # 리스트 중 유사도 가장 낮은단어 반환

'lie'